## Part 2

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
filename_data = 'mycsv.csv'
csv = spark.read.csv(filename_data, inferSchema=True, header=True)
csv = csv.drop(csv._c0).withColumn('label', when(col('price') >= 4212274, 1).otherwise(0))
csv.drop(csv['price']).show(10)

+-----+------+-----+----+-------------+-----------+---------+-----+
|level|levels|rooms|area|building_type|object_type|id_region|label|
+-----+------+-----+----+-------------+-----------+---------+-----+
|   15|    31|    1|30.3|            0|          2|       66|    0|
|    5|     5|    1|33.0|            0|          0|        1|    0|
|    4|    13|    3|85.0|            3|          0|       50|    1|
|    3|     5|    3|82.0|            0|          0|        1|    0|
|    2|     3|    1|30.0|            3|          2|       23|    0|
|    5|     5|    2|47.0|            2|          0|       34|    0|
|    1|     2|    3|54.0|            0|          0|        3|    0|
|   16|    18|    1|39.7|            4|          0|       23|    0|
|    2|     2|    2|53.2|            5|          0|       58|    0|
|    1|    19|   -1|10.1|            3|          0|       77|    0|
+-----+------+-----+----+-------------+-----------+---------+-----+
only showing top 10 rows



In [3]:
splits = csv.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 4622905  Testing Rows: 1979846


In [4]:
numVect = VectorAssembler(inputCols = ['building_type', 'rooms', 'level'], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
featVect = VectorAssembler(inputCols=["normFeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label", 
                        featuresCol="features", 
                        maxIter=10,
                        regParam=0.3)
pipeline = Pipeline(stages=[numVect, minMax, featVect, lr])

In [5]:
pipelineModel = pipeline.fit(train)

In [6]:
prediction = pipelineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+-------------------------------------------------------------+----------+---------+
|features                                                     |prediction|trueLabel|
+-------------------------------------------------------------+----------+---------+
|[0.6666666666666666,0.30000000000000004,0.0]                 |0.0       |0        |
|[0.3333333333333333,0.0,0.022222222222222223]                |0.0       |0        |
|[0.0,0.30000000000000004,0.0]                                |0.0       |0        |
|[0.3333333333333333,0.30000000000000004,0.0]                 |0.0       |0        |
|[0.6666666666666666,0.2,0.0]                                 |0.0       |0        |
|[0.6666666666666666,0.30000000000000004,0.06666666666666667] |0.0       |0        |
|[0.0,0.2,0.0]                                                |0.0       |0        |
|[0.3333333333333333,0.30000000000000004,0.022222222222222223]|0.0       |0        |
|[0.0,0.2,0.0]                                                |0.

In [7]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|            49906.0|
|       FP|            14940.0|
|       TN|          1234749.0|
|       FN|           680251.0|
|Precision| 0.7696079943250161|
|   Recall|0.06834968369816355|
|       F1|0.12554921176398076|
+---------+-------------------+



In [8]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR = ", aur)

AUR =  0.6844462051239985


In [9]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.1]).addGrid(lr.maxIter, [10, 5]).build()

cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(metricName='areaUnderPR'), estimatorParamMaps=paramGrid, 
                    numFolds=2)
model = cv.fit(train)

In [10]:
newPrediction = model.transform(test)
newPredicted = prediction.select("features", "prediction", "trueLabel")
newPredicted.show()

+--------------------+----------+---------+
|            features|prediction|trueLabel|
+--------------------+----------+---------+
|[0.66666666666666...|       0.0|        0|
|[0.33333333333333...|       0.0|        0|
|[0.0,0.3000000000...|       0.0|        0|
|[0.33333333333333...|       0.0|        0|
|[0.66666666666666...|       0.0|        0|
|[0.66666666666666...|       0.0|        0|
|       [0.0,0.2,0.0]|       0.0|        0|
|[0.33333333333333...|       0.0|        0|
|       [0.0,0.2,0.0]|       0.0|        0|
|[0.66666666666666...|       0.0|        0|
|[0.83333333333333...|       0.0|        0|
|[0.0,0.3000000000...|       0.0|        0|
|[0.0,0.4,0.066666...|       0.0|        0|
|[0.0,0.2,0.066666...|       0.0|        0|
|[0.33333333333333...|       0.0|        0|
|[0.0,0.3000000000...|       0.0|        0|
|[0.66666666666666...|       0.0|        0|
|[0.33333333333333...|       0.0|        0|
|[0.66666666666666...|       0.0|        0|
|[0.0,0.2,0.333333...|       0.0

In [11]:
# Recalculate confusion matrix
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
pr2 = tp2 / (tp2 + fp2)
re2 = tp2 / (tp2 + fn2)
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", pr2),
 ("Recall", re2),
 ("F1", 2*pr2*re2/(re2+pr2))],["metric", "value"])
metrics2.show()

+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|           138401.0|
|       FP|            66451.0|
|       TN|          1183238.0|
|       FN|           591756.0|
|Precision| 0.6756145900454963|
|   Recall| 0.1895496448024192|
|       F1|0.29604207018328166|
+---------+-------------------+



In [12]:
# Recalculate the Area Under ROC
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator.evaluate(prediction)
print( "AUR2 = ", aur2)

AUR2 =  0.6844462051239985


## Part 1

In [23]:
splits = csv.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("price", "truePrice")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 4622110  Testing Rows: 1980641


In [25]:
numVect = VectorAssembler(inputCols = ['building_type', 'rooms', 'level'], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
featVect = VectorAssembler(inputCols=["normFeatures"], outputCol="features")
lr = RandomForestRegressor(labelCol="price", 
                        featuresCol="features", 
                        numTrees = 10,
                      maxDepth=2,
                      maxBins = 10000)
pipeline = Pipeline(stages=[numVect, minMax, featVect, lr])

In [26]:
pipelineModel = pipeline.fit(train)

In [27]:
prediction = pipelineModel.transform(test)
predicted = prediction.select("features", "prediction", "truePrice")
predicted.show(100, truncate=False)

+-------------------------------------------------------------+------------------+---------+
|features                                                     |prediction        |truePrice|
+-------------------------------------------------------------+------------------+---------+
|[0.6666666666666666,0.30000000000000004,0.0]                 |3623610.9642504463|1        |
|[0.3333333333333333,0.4,0.0]                                 |3993699.7354937913|1        |
|[0.3333333333333333,0.2,0.022222222222222223]                |3561868.3618761064|550      |
|[0.3333333333333333,0.30000000000000004,0.0]                 |3956291.3495297185|600      |
|[0.3333333333333333,0.30000000000000004,0.022222222222222223]|4013462.3349721944|650      |
|[0.8333333333333333,0.30000000000000004,0.044444444444444446]|3680781.949692922 |1150     |
|[0.6666666666666666,0.2,0.06666666666666667]                 |3319602.130896868 |1250     |
|[0.3333333333333333,0.4,0.022222222222222223]                |4050870

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="truePrice", metricName="rmse")

In [33]:
# RMSE
rmse = regressionEvaluator.evaluate(predicted)
print(f"The RMSE for the random forest regression model is {rmse:0.2f}")
# MSE
mse = regressionEvaluator.setMetricName("mse").evaluate(predicted)
print(f"The MSE for the random forest regression model is {mse:0.2f}")
# R2
r2 = regressionEvaluator.setMetricName("r2").evaluate(predicted)
print(f"The R2 for the random forest regression model is {r2:0.2f}")
# MAE
mae = regressionEvaluator.setMetricName("mae").evaluate(predicted)
print(f"The MAE for the random forest regression model is {mae:0.2f}")

The RMSE for the random forest regression model is 5123342944930.38
The MSE for the random forest regression model is 5123342944930.38
The R2 for the random forest regression model is 0.10
The MAE for the random forest regression model is 1737081.15


In [36]:
param_grid = ParamGridBuilder().\
    addGrid(lr.numTrees, [10, 15, 20]).\
    addGrid(lr.maxDepth, [1, 2, 4]).\
    addGrid(lr.maxBins , [10000, 20000, 30000]).\
    build()

In [37]:
cv = CrossValidator(estimator=pipeline, \
                    estimatorParamMaps=param_grid, \
                    evaluator=RegressionEvaluator(
                                predictionCol="prediction", \
                                labelCol="price", \
                                metricName="rmse"), \
                    numFolds=2)

In [38]:
cv_model = cv.fit(train)

In [39]:
newPrediction = cv_model.transform(test)

In [40]:
# RMSE
rmse = regressionEvaluator.evaluate(newPrediction)
print(f"The RMSE for the random forest regression model is {rmse:0.2f}")
# MSE
mse = regressionEvaluator.setMetricName("mse").evaluate(newPrediction)
print(f"The MSE for the random forest regression model is {mse:0.2f}")
# R2
r2 = regressionEvaluator.setMetricName("r2").evaluate(newPrediction)
print(f"The R2 for the random forest regression model is {r2:0.2f}")
# MAE
mae = regressionEvaluator.setMetricName("mae").evaluate(newPrediction)
print(f"The MAE for the random forest regression model is {mae:0.2f}")

The RMSE for the random forest regression model is 1673747.78
The MSE for the random forest regression model is 4811850009048.07
The R2 for the random forest regression model is 0.15
The MAE for the random forest regression model is 1673747.78
